<a href="https://colab.research.google.com/github/robinyUArizona/PyTorch-Deep-Learning/blob/main/simple_NeuralNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>